In [1]:
import requests
import time
import json
from datetime import datetime, timedelta, timezone

url = "https://app.birdweather.com/graphql"  # GraphQL API Endpoint
download_interval = 30  # Wartezeit zwischen den Abfragen in Sekunden

# Zeitzone +02:00 definieren
tz = timezone(timedelta(hours=2))

# Bestimme das heutige Datum (in der angegebenen Zeitzone)
now = datetime.now(tz)
today_date = now.date()

# Startdatum: 3 Wochen (21 Tage) vor heute
start_date = today_date - timedelta(days=21)

def generate_day_range(day):
    """
    Erzeugt für einen gegebenen Tag den Startzeitpunkt (18:00 Uhr)
    und den Endzeitpunkt (18:00 Uhr am Folgetag) im ISO-Format.
    """
    start = datetime(day.year, day.month, day.day, 18, 0, 0, tzinfo=tz)
    end = start + timedelta(days=1)
    return start, end

# Liste zur Sammlung aller "edges"-Elemente aus den Antworten
all_edges = []

current_date = start_date
while current_date <= today_date:
    start, end = generate_day_range(current_date)
    
    # Erstelle die Query mit den dynamisch eingefügten Zeitpunkten
    query = f"""
query Detections {{
    detections(
        stationIds: ["8420"]
        period: {{ from: "{start.isoformat()}", to: "{end.isoformat()}" }}
    ) {{
        speciesCount
        totalCount
        edges {{
            node {{
                certainty
                confidence
                eclipse
                id
                probability
                score
                speciesId
                timestamp
                species {{
                    commonName
                    id
                    imageUrl
                    scientificName
                    thumbnailUrl
                    color
                }}
            }}
        }}
    }}
}}
    """
    try:
        response = requests.post(url, json={'query': query})
        if response.status_code == 200:
            result = response.json()
            # Extrahiere die "edges" aus der Antwort (falls vorhanden)
            edges = result.get("data", {}).get("detections", {}).get("edges", [])
            if edges:
                all_edges.extend(edges)
                print(f"Daten für {start.isoformat()} bis {end.isoformat()} erfolgreich hinzugefügt.")
            else:
                print(f"Keine 'edges'-Daten für {start.isoformat()} bis {end.isoformat()} gefunden.")
        else:
            print(f"Fehler: Server antwortete mit Status-Code {response.status_code}")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")
    
    time.sleep(download_interval)
    current_date += timedelta(days=1)

# Nach der Schleife alle gesammelten "edges" in eine JSON-Datei schreiben
with open("downloaded3_data.json", "w", encoding="utf-8") as f:
    json.dump(all_edges, f, indent=2, ensure_ascii=False)

print("Alle 'edges'-Daten wurden in downloaded_data.json gespeichert.")

Daten für 2025-03-15T18:00:00+02:00 bis 2025-03-16T18:00:00+02:00 erfolgreich hinzugefügt.
Daten für 2025-03-16T18:00:00+02:00 bis 2025-03-17T18:00:00+02:00 erfolgreich hinzugefügt.
Daten für 2025-03-17T18:00:00+02:00 bis 2025-03-18T18:00:00+02:00 erfolgreich hinzugefügt.
Keine 'edges'-Daten für 2025-03-18T18:00:00+02:00 bis 2025-03-19T18:00:00+02:00 gefunden.
Daten für 2025-03-19T18:00:00+02:00 bis 2025-03-20T18:00:00+02:00 erfolgreich hinzugefügt.
Keine 'edges'-Daten für 2025-03-20T18:00:00+02:00 bis 2025-03-21T18:00:00+02:00 gefunden.
Daten für 2025-03-21T18:00:00+02:00 bis 2025-03-22T18:00:00+02:00 erfolgreich hinzugefügt.
Daten für 2025-03-22T18:00:00+02:00 bis 2025-03-23T18:00:00+02:00 erfolgreich hinzugefügt.
Daten für 2025-03-23T18:00:00+02:00 bis 2025-03-24T18:00:00+02:00 erfolgreich hinzugefügt.
Daten für 2025-03-24T18:00:00+02:00 bis 2025-03-25T18:00:00+02:00 erfolgreich hinzugefügt.
Daten für 2025-03-25T18:00:00+02:00 bis 2025-03-26T18:00:00+02:00 erfolgreich hinzugefügt.
D